In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy
from datetime import datetime
import time
from sklearn.preprocessing import LabelEncoder

In [2]:
## load data
train = pd.read_csv('products.csv')
test = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)


In [4]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [5]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

train.replace({"State": state_dict}, inplace=True)

In [6]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(200)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [7]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [8]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'Gender', 'State', 'transactionDate', 'store_code','transaction_number_by_till','product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [9]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

,customerID,Gender,State,transactionDate,store_code,transaction_number_by_till,product_code
0,BBID_2041,male,no_state,2016-03-20,2655.0,72624.0,1.081e+08
1,BBID_2041,male,no_state,2016-03-20,2655.0,72624.0,1.081e+08
2,BBID_2041,male,no_state,2016-03-20,2655.0,72624.0,1.081e+08
3,BBID_2041,male,no_state,2016-03-20,2655.0,72624.0,1.081e+08
4,BBID_2041,male,no_state,2016-03-20,2655.0,72624.0,1.081e+08


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7981248 entries, 0 to 7981247
Data columns (total 7 columns):
customerID                    object
Gender                        object
State                         object
transactionDate               object
store_code                    float64
transaction_number_by_till    float64
product_code                  object
dtypes: float64(2), object(5)
memory usage: 487.1+ MB


In [16]:
## convert data into numeric / float

for c in train.columns:
    lbl = LabelEncoder()
    if train[c].dtype != 'object' and c not in ['customerID','transactionDate', 'Gender', 'State', 'product_code']:
        train[c] = lbl.fit_transform(train[c])

In [17]:
train.head()

,customerID,Gender,State,transactionDate,store_code,transaction_number_by_till,product_code
0,BBID_2041,male,no_state,2016-03-20,1,56777,1.081e+08
1,BBID_2041,male,no_state,2016-03-20,1,56777,1.081e+08
2,BBID_2041,male,no_state,2016-03-20,1,56777,1.081e+08
3,BBID_2041,male,no_state,2016-03-20,1,56777,1.081e+08
4,BBID_2041,male,no_state,2016-03-20,1,56777,1.081e+08


In [18]:
train.dropna(inplace=True)

In [19]:
train['product_code'] = train['product_code'].astype(int)

In [20]:
prod_dict = {'_'}

train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

,customerID,Gender,State,transactionDate,storecode,transactionnumberbytill,100105505,108000537,108000568,108000599,...,1000059431,1000067678,1000088419,1000088500,1000123958,1000171548,1000336252,1000609658,1000638372,1000694851
0,BBID_2041,male,no_state,2016-03-20,1,56777,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_2041,male,no_state,2016-03-20,1,56777,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_2041,male,no_state,2016-03-20,1,56777,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_2041,male,no_state,2016-03-20,1,56777,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_2041,male,no_state,2016-03-20,1,56777,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_train = train.copy()

In [22]:
df_june = df_train[df_train['transactionDate'] == '2017-06-28']

In [23]:
df_june.reset_index(drop=True)

,customerID,Gender,State,transactionDate,storecode,transactionnumberbytill,100105505,108000537,108000568,108000599,...,1000059431,1000067678,1000088419,1000088500,1000123958,1000171548,1000336252,1000609658,1000638372,1000694851
0,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,BBID_20470,male,no_state,2017-06-28,2,51389,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
pd.set_option('display.max_columns', None)

In [25]:
columns = []

for row in df_june:
    columns.append(row)

In [27]:
product_col = ['100105505', '108000537', '108000568', '108000599', '108000653',
 '108000655', '108000656', '108000667', '108000704', '108000707', '108001065',
 '108001125', '108001127', '108001179', '108001180', '108001187', '108003214',
 '108003448', '108003451', '108003545', '108004398', '108004423', '108004507',
 '108004511', '108004526', '108004590', '108004629', '108004632', '108004633',
 '108004880', '108004977', '108005183', '108005676', '108005681', '108008866',
 '108010225', '108015124', '108015133', '108015369', '108016183', '108016186',
 '108016189', '108017242', '108017243', '108017245', '108017697', '108018729',
 '108018730', '108018750', '108018925', '108020370', '108020392', '108020412', 
 '108020414', '108020418', '108020419', '108020428', '108020429', '108020477',
 '108020517', '108020526', '108027348', '108027409', '108030063', '108030064',
 '108030066', '108030093', '108030095', '108030108', '108030113', '108032048',
 '108032050', '108032209', '108035243', '108037489', '108037499', '108037502',
 '108037504', '108037506', '108037568', '108043278', '108100064', '108100137',
 '108100143', '108100145', '108100162', '108100183', '108100193', '108100213',
 '108100221', '108100225', '108100243', '108100253', '108100261', '108100267',
 '108100272', '108100277', '108100278', '108100280', '108100288', '108100290',
 '108100294', '108100296', '108100297', '108100298', '108100302', '108100306',
 '108100308', '108100319', '108100325', '108100328', '108100330', '108100331',
 '108100334', '108100335', '108100340', '108100352', '108100361', '108100362',
 '108100373', '108100379', '108100382', '300027193', '300028156', '300043603',
 '300067574', '300070015', '300070102', '300077074', '300088564', '300111517',
 '300143077', '300144423', '300150379', '300150382', '300157247', '300165587',
 '300177372', '300196348', '300251241', '300317078', '300326493', '300347917',
 '300356444', '300361324', '300374464', '300481673', '300481738', '300481740',
 '300481741', '300491428', '300497842', '300511394', '300520934', '300528552',
 '300572518', '300600437', '300600438', '300616595', '300647581', '300676075',
 '300680079', '300685217', '300702014', '300776408', '300776409', '300776410',
 '300776411', '300781593', '300785123', '300785147', '300785148', '300785150',
 '300808923', '300812356', '300840018', '300930107', '300942704', '300956483',
 '300974316', '300974360', '300981554', '300981880', '300981918', '300989165',
 '500096181', '530000111', '1000005980', '1000010411', '1000058092', '1000059431',
 '1000067678', '1000088419', '1000088500', '1000123958', '1000171548', '1000336252',
 '1000609658', '1000638372', '1000694851']

In [28]:
month = 2

df_train_5 = df_train.loc[df_train['transactionDate']=='2017-05-28', product_col+['customerID']]
df_train_4 = df_train.loc[df_train['transactionDate']=='2017-04-28', product_col+['customerID']]
#df_train_3 = df_train.loc[df_train['transactionDate']=='2017-03-28', product_col+['customerID']]
#df_train_2 = df_train.loc[df_train['transactionDate']=='2017-02-28', product_col+['customerID']]
#df_train_1 = df_train.loc[df_train['transactionDate']=='2017-01-28', product_col+['customerID']]

In [29]:
dfm = pd.merge(df_june,df_train_5, how='left', on=['customerID'], suffixes=('', '_5'))
dfm = pd.merge(dfm,df_train_4, how='left', on=['customerID'], suffixes=('', '_4'))

In [30]:
all_product_col = [col for col in dfm.columns if '' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [31]:
dfm.head()

,customerID,Gender,State,transactionDate,storecode,transactionnumberbytill,100105505,108000537,108000568,108000599,108000653,108000655,108000656,108000667,108000704,108000707,108001065,108001125,108001127,108001179,108001180,108001187,108003214,108003448,108003451,108003545,108004398,108004423,108004507,108004511,108004526,108004590,108004629,108004632,108004633,108004880,108004977,108005183,108005676,108005681,108008866,108010225,108015124,108015133,108015369,108016183,108016186,108016189,108017242,108017243,108017245,108017697,108018729,108018730,108018750,108018925,108020370,108020392,108020412,108020414,108020418,108020419,108020428,108020429,108020477,108020517,108020526,108027348,108027409,108030063,108030064,108030066,108030093,108030095,108030108,108030113,108032048,108032050,108032209,108035243,108037489,108037499,108037502,108037504,108037506,108037568,108043278,108100064,108100137,108100143,108100145,108100162,108100183,108100193,108100213,108100221,108100225,108100243,108100253,108100261,108100267,108100272,108100277,108100278,108100280,108100288,108100290,108100294,108100296,108100297,108100298,108100302,108100306,108100308,108100319,108100325,108100328,108100330,108100331,108100334,108100335,108100340,108100352,108100361,108100362,108100373,108100379,108100382,300027193,300028156,300043603,300067574,300070015,300070102,300077074,300088564,300111517,300143077,300144423,300150379,300150382,300157247,300165587,300177372,300196348,300251241,300317078,300326493,300347917,300356444,300361324,300374464,300481673,300481738,300481740,300481741,300491428,300497842,300511394,300520934,300528552,300572518,300600437,300600438,300616595,300647581,300676075,300680079,300685217,300702014,300776408,300776409,300776410,300776411,300781593,300785123,300785147,300785148,300785150,300808923,300812356,300840018,300930107,300942704,300956483,300974316,300974360,300981554,300981880,300981918,300989165,500096181,530000111,1000005980,1000010411,1000058092,1000059431,1000067678,1000088419,1000088500,1000123958,1000171548,1000336252,1000609658,1000638372,1000694851,100105505_5,108000537_5,108000568_5,108000599_5,108000653_5,108000655_5,108000656_5,108000667_5,108000704_5,108000707_5,108001065_5,108001125_5,108001127_5,108001179_5,108001180_5,108001187_5,108003214_5,108003448_5,108003451_5,108003545_5,108004398_5,108004423_5,108004507_5,108004511_5,108004526_5,108004590_5,108004629_5,108004632_5,108004633_5,108004880_5,108004977_5,108005183_5,108005676_5,108005681_5,108008866_5,108010225_5,108015124_5,108015133_5,108015369_5,108016183_5,108016186_5,108016189_5,108017242_5,108017243_5,108017245_5,108017697_5,108018729_5,108018730_5,108018750_5,108018925_5,108020370_5,108020392_5,108020412_5,108020414_5,108020418_5,108020419_5,108020428_5,108020429_5,108020477_5,108020517_5,108020526_5,108027348_5,108027409_5,108030063_5,108030064_5,108030066_5,108030093_5,108030095_5,108030108_5,108030113_5,108032048_5,108032050_5,108032209_5,108035243_5,108037489_5,108037499_5,108037502_5,108037504_5,108037506_5,108037568_5,108043278_5,108100064_5,108100137_5,108100143_5,108100145_5,108100162_5,108100183_5,108100193_5,108100213_5,108100221_5,108100225_5,108100243_5,108100253_5,108100261_5,108100267_5,108100272_5,108100277_5,108100278_5,108100280_5,108100288_5,108100290_5,108100294_5,108100296_5,108100297_5,108100298_5,108100302_5,108100306_5,108100308_5,108100319_5,108100325_5,108100328_5,108100330_5,108100331_5,108100334_5,108100335_5,108100340_5,108100352_5,108100361_5,108100362_5,108100373_5,108100379_5,108100382_5,300027193_5,300028156_5,300043603_5,300067574_5,300070015_5,300070102_5,300077074_5,300088564_5,300111517_5,300143077_5,300144423_5,300150379_5,300150382_5,300157247_5,300165587_5,300177372_5,300196348_5,300251241_5,300317078_5,300326493_5,300347917_5,300356444_5,300361324_5,300374464_5,300481673_5,300481738_5,300481740_5,300481741_5,300491428_5,300497842_5,300511394_5,300520934_5,300528552_5,300572518_5,300600437_5,300600438_5,300616595_5,3006

In [32]:
for col in product_col:
    dfm[col+'_5'] =dfm[col]
    
dfm.drop(product_col, axis=1, inplace=True)

In [33]:
dfm.drop('transactionDate', axis=1, inplace=True)

In [34]:
dfm.to_csv('cleaned_data/TestSet_withpast3_new2_woDecimal_200products_newcolumns.csv', index=False)